In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install natsort

In [3]:
import torch
print("Using torch", torch.__version__)
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import TransformerEncoder, TransformerEncoderLayer
import os
import cv2
import natsort

Using torch 2.3.0+cu121


In [ ]:
import struct
import sys
import filters
import math
import argparse

In [36]:
# filter the data to remove the sections in the video where the force threshold hasn't yet been met

def filter_csv_data(input_wrench_file, output_wrench_file, input_joint_file, output_joint_file, threshold=-1.4):
    # Load the CSV file
    wrench_data = pd.read_csv(input_wrench_file)
    joint_data = pd.read_csv(input_joint_file)

    window_size = 20 # number of values to take the average of
    tolerance = 0.01

    # Find the first occurrence where average force_x exceeds the threshold or is within tolerance of the threshold
    force_x_avg = wrench_data['force_x'].rolling(window=window_size).mean()
    print("size: ", force_x_avg.size)
    condition_met = ((np.abs(force_x_avg - threshold) <= 0.01) | (force_x_avg > threshold))

    if condition_met.any():
        index_threshold = condition_met.idxmax()
        print("index_threshold: ", index_threshold)
    else:
        # raise ValueError("No data point meets the specified condition.")
        print("No data point meets the specified condition.")

    # ------------------------------------------------------------------
    # # Keep all rows after this index
    # filtered_wrench_data = wrench_data.iloc[index_threshold:]
    # filtered_joint_data = joint_data.iloc[index_threshold:]

    # # Save the filtered data to a new CSV file
    # filtered_wrench_data.to_csv(output_wrench_file, index=False)
    # filtered_joint_data.to_csv(output_joint_file, index=False)

    # return index_threshold

def process_video(input_video, output_video, start_frame):
    # Open the input video
    cap = cv2.VideoCapture(input_video)

    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Write frames starting from the start_frame
        if frame_count >= start_frame:
            out.write(frame)

        frame_count += 1

    print(f'Frames of input video: {frame_count}')
    print(f'Frames of output video: {frame_count - start_frame}')

    # Release everything if job is finished
    cap.release()
    out.release()

def process_files(calibrated_video_dir, calibrated_video_output_dir, uncalibrated_video_dir, uncalibrated_video_output_dir, wrench_dir, wrench_output_dir, joint_dir, joint_output_dir, wrench_prefix='wrench_data_', video_prefix='output_', output_video_prefix='filtered_video_', joint_prefix='joint_states_', threshold=-1.4):
    # Ensure output directories exist
    os.makedirs(calibrated_video_output_dir, exist_ok=True)
    os.makedirs(uncalibrated_video_output_dir, exist_ok=True)
    os.makedirs(wrench_output_dir, exist_ok=True)
    os.makedirs(joint_output_dir, exist_ok=True)

    print("video directory: ", calibrated_video_dir)

    for file_name in os.listdir(calibrated_video_dir):
        trial_num = int(file_name.split('_')[1].split('.')[0])
        input_calibrated_video = os.path.join(calibrated_video_dir, f'{video_prefix}{trial_num}.avi')
        input_uncalibrated_video = os.path.join(uncalibrated_video_dir, f'{video_prefix}{trial_num}.avi')
        input_wrench_csv = os.path.join(wrench_dir, f'{wrench_prefix}{trial_num}.csv')
        input_joint_csv = os.path.join(joint_dir, f'{joint_prefix}{trial_num}.csv')

        if not os.path.exists(input_calibrated_video) or not os.path.exists(input_uncalibrated_video) or not os.path.exists(input_wrench_csv) or not os.path.exists(input_joint_csv):
            print(f'Files not found: {input_calibrated_video}, {input_uncalibrated_video}, {input_wrench_csv}, {input_joint_csv}')
            break

        output_calibrated_video = os.path.join(calibrated_video_output_dir, f'{output_video_prefix}{trial_num}.avi')
        output_uncalibrated_video = os.path.join(uncalibrated_video_output_dir, f'{output_video_prefix}{trial_num}.avi')
        output_wrench_csv = os.path.join(wrench_output_dir, f'filtered_{wrench_prefix}{trial_num}.csv')
        output_joint_csv = os.path.join(joint_output_dir, f'filtered_{joint_prefix}{trial_num}.csv')

        start_frame = filter_csv_data(input_wrench_csv, output_wrench_csv, input_joint_csv, output_joint_csv, threshold)
        print("start_frame: ", start_frame)
        print(f'Processed {input_wrench_csv} -> {output_wrench_csv} and {input_joint_csv} -> {output_joint_csv}')
        # process_video(input_calibrated_video, output_calibrated_video, start_frame)
        # process_video(input_uncalibrated_video, output_uncalibrated_video, start_frame)
        # print(f'Processed {input_calibrated_video} -> {output_calibrated_video}')
        # print(f'Processed {input_uncalibrated_video} -> {output_uncalibrated_video}')

data_path = '/content/drive/MyDrive/soft_manipulation_videos/final_data_for_paper'
# layer_str_arr = ['0_layer', '1_layer','2_layer', '3_layer']
layer_str_arr = ['0_layer']
for layer_str in layer_str_arr:
    calibrated_video_dir = os.path.join(data_path, f'raw_data/output_videos/calibrated_cam/{layer_str}')
    uncalibrated_video_dir = os.path.join(data_path, f'raw_data/output_videos/uncalibrated_cam/{layer_str}')
    wrench_dir = os.path.join(data_path, f'raw_data/wrench_data/{layer_str}')
    joint_dir = os.path.join(data_path, f'raw_data/joint_state_data/{layer_str}')

    calibrated_video_output_dir = os.path.join(data_path, f'filtered_data/video_data/calibrated_cam/{layer_str}')
    uncalibrated_video_output_dir = os.path.join(data_path, f'filtered_data/video_data/uncalibrated_cam/{layer_str}')
    wrench_output_dir = os.path.join(data_path, f'filtered_data/wrench_data/{layer_str}')
    joint_output_dir = os.path.join(data_path, f'filtered_data/joint_data/{layer_str}')
    process_files(calibrated_video_dir, calibrated_video_output_dir, uncalibrated_video_dir, uncalibrated_video_output_dir, wrench_dir, wrench_output_dir, joint_dir, joint_output_dir)


video directory:  /content/drive/MyDrive/soft_manipulation_videos/final_data_for_paper/raw_data/output_videos/calibrated_cam/0_layer
size:  240
index_threshold:  19
start_frame:  None
Processed /content/drive/MyDrive/soft_manipulation_videos/final_data_for_paper/raw_data/wrench_data/0_layer/wrench_data_0.csv -> /content/drive/MyDrive/soft_manipulation_videos/final_data_for_paper/filtered_data/wrench_data/0_layer/filtered_wrench_data_0.csv and /content/drive/MyDrive/soft_manipulation_videos/final_data_for_paper/raw_data/joint_state_data/0_layer/joint_states_0.csv -> /content/drive/MyDrive/soft_manipulation_videos/final_data_for_paper/filtered_data/joint_data/0_layer/filtered_joint_states_0.csv
size:  239
index_threshold:  19
start_frame:  None
Processed /content/drive/MyDrive/soft_manipulation_videos/final_data_for_paper/raw_data/wrench_data/0_layer/wrench_data_1.csv -> /content/drive/MyDrive/soft_manipulation_videos/final_data_for_paper/filtered_data/wrench_data/0_layer/filtered_wrench

In [ ]:
# # Save NPZ file for flow data
# print(flow_data.shape)
# # np.savez('/Users/ankushdhawan/Documents/Stanford/Research/ARM Lab/soft_manipulation/data/2layer_trials_alldata_small.npz', flow_data=flow_data)
# np.savez('/content/drive/MyDrive/soft_manipulation_videos/final_data/1layer_trials_alldata_avgpoolby6.npz', flow_data=flow_data)

In [ ]:
# determine the optical flow

num_trials = 300
num_channels = 2
num_sampled_frames = 23
height = 768
width = 1024

def denseOpticalFlow(file_path, filename="output", num_sampled_frames=10, num_channels=2):

    # define a video capture object
    cap = cv2.VideoCapture(file_path)

    # determine the total number of available frames in the video
    total_num_frames = 0
    saved_frames = []  # had to save frames since there was an issue with re-reading frames when calculating optical flow
    while True:
        ret, frame = cap.read()
        if not ret:  # frame is not valid
            break
        total_num_frames += 1
        saved_frames.append(frame)

    # sample frames
    sampled_frame_indices = np.linspace(0, total_num_frames - 1, num_sampled_frames, dtype=int)

    # Create random colors
    color = np.random.randint(0, 255, (300, 3))
    old_gray = cv2.cvtColor(saved_frames[0], cv2.COLOR_BGR2GRAY)

    first_flag = 0  # 0 if it is the first time in the loop, 1 otherwise
    for frame_idx in sampled_frame_indices:
        frame = saved_frames[frame_idx]

        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Calculates dense optical flow by Farneback method
        flow = cv2.calcOpticalFlowFarneback(old_gray, frame_gray,
                                            None,
                                            0.5,
                                            2,
                                            8,
                                            2,
                                            5,
                                            1.1,
                                            0)

        # Computes the magnitude and angle of the 2D vectors
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        height = mag.shape[0]
        width = mag.shape[1]
        mag = mag[np.newaxis, np.newaxis, :, :]
        ang = ang[np.newaxis, np.newaxis, :, :]

        if first_flag == 0:  # first frame
            curr_flow_vals = np.zeros((num_channels, 1, height, width))
            all_flow_vals = np.zeros((num_channels, 1, height, width))  # shape of flow vals: (num_channels (2), depth (num_sampled_frames), height, width). 0th channel has magnitudes, 1st channel has angles

        curr_flow_vals[0, 0, :, :] = mag
        curr_flow_vals[1, 0, :, :] = ang

        if first_flag == 0:
            all_flow_vals = curr_flow_vals
        else:
            all_flow_vals = np.concatenate((all_flow_vals, curr_flow_vals), axis=1)

        old_gray = frame_gray.copy()

        first_flag = 1

    cap.release()
    cv2.destroyAllWindows()

    # Apply average pooling with a neighborhood of 36
    pooled_flow_vals = np.zeros((num_channels, all_flow_vals.shape[1] // 36, height // 6, width // 6))

    for i in range(0, all_flow_vals.shape[1], 36):
        for j in range(0, height, 6):
            for k in range(0, width, 6):
                mag_patch = all_flow_vals[0, i:i+36, j:j+6, k:k+6]
                ang_patch = all_flow_vals[1, i:i+36, j:j+6, k:k+6]

                pooled_flow_vals[0, i//36, j//6, k//6] = np.mean(mag_patch)
                pooled_flow_vals[1, i//36, j//6, k//6] = np.mean(ang_patch)

    return pooled_flow_vals


def main(folder_path):

    if not os.path.isdir(folder_path):
        print(f"The path '{folder_path}' is not a directory.")
        return

    # flow_data = np.zeros((num_trials, num_channels, num_sampled_frames, height, width))
    flow_data = None

    # Loop through each file in each folder in the directory
    # for entry in os.listdir(folder_path): # for each folder (0_layer, 1_layer, 2_layer)
    for entry in natsort.natsorted(os.listdir(folder_path)): # for each folder (0_layer, 1_layer, 2_layer)
        print("folder name: ", entry)
        # if entry != "desktop.ini": # make sure it's not desktop.ini, which is an artifact from using Google Drive desktop
        if entry == "2_layer":
            subfolder_path = os.path.join(folder_path, entry)
            num_layers = int(entry.split('_')[0])
            # print(num_layers)
            for file_name in natsort.natsorted(os.listdir(subfolder_path)):
                file_path = os.path.join(subfolder_path, file_name) # Get the full path of the file
                if os.path.isfile(file_path) and (file_name != "desktop.ini"): # Check if the path is a file (not a directory). make sure it's not desktop.ini, which is an artifact from using Google Drive desktop
                    print("file name: ", file_name)
                    trial_num = int(file_name.split('_')[2].split('.')[0]) # file_name format: filtered_video_20.avi
                    # print("trial_num: ", trial_num)
                    if (trial_num >= 0) and (trial_num <= 25) : # only do 33 trials
                        # print("trial_num: ", trial_num)
                        trial_flow_vals = denseOpticalFlow(file_path, file_name) # all flow values for a single trial
                        trial_flow_vals = np.expand_dims(trial_flow_vals, axis=0)
                        if flow_data is None:
                            flow_data = trial_flow_vals
                        else:
                            flow_data = np.concatenate((flow_data, trial_flow_vals), axis=0)
                        print("flow data shape: ", flow_data.shape)
                        if (trial_num == 32) :
                            break
                    # break
            # break

    # print("length of list = ", len(flow_data))
    return flow_data

data_path = 'G:/.shortcut-targets-by-id/1IrduQonbo3U34ys_7cPgPFa4pvZBiA44/soft_manipulation_videos/tests_5_28/filtered_test_data_5_28/video_data'
# data_path = '/content/drive/MyDrive/soft_manipulation_videos/tests_5_28/filtered_test_data_5_28/video_data'
# main(data_path)
flow_data = main(data_path)
# print(flow_data.shape)


folder name:  0_layer
folder name:  1_layer
folder name:  2_layer
file name:  filtered_video_0.avi
flow data shape:  (1, 2, 23, 768, 1024)
file name:  filtered_video_1.avi
flow data shape:  (2, 2, 23, 768, 1024)
file name:  filtered_video_2.avi
flow data shape:  (3, 2, 23, 768, 1024)
file name:  filtered_video_3.avi
flow data shape:  (4, 2, 23, 768, 1024)
file name:  filtered_video_4.avi
flow data shape:  (5, 2, 23, 768, 1024)
file name:  filtered_video_5.avi
flow data shape:  (6, 2, 23, 768, 1024)
file name:  filtered_video_6.avi
flow data shape:  (7, 2, 23, 768, 1024)
file name:  filtered_video_7.avi
flow data shape:  (8, 2, 23, 768, 1024)
file name:  filtered_video_8.avi
flow data shape:  (9, 2, 23, 768, 1024)
file name:  filtered_video_9.avi
flow data shape:  (10, 2, 23, 768, 1024)
file name:  filtered_video_10.avi
flow data shape:  (11, 2, 23, 768, 1024)
file name:  filtered_video_11.avi
flow data shape:  (12, 2, 23, 768, 1024)
file name:  filtered_video_12.avi
flow data shape:  (

In [ ]:
reshaped_flow_data = flow_data.reshape(-1, height*width) # num_trials*num_channels*num_sampled_frames by height*width
print(reshaped_flow_data.shape)
np.savetxt("C:/Users/chung/OneDrive - Stanford/2023-2024/ARMLab Research/Spring/optical_flow_saves/2_layer_trials0to32.csv", reshaped_flow_data, delimiter=",")

(1518, 786432)


In [ ]:
loaded_flow_data = np.loadtxt('C:/Users/chung/OneDrive - Stanford/2023-2024/ARMLab Research/Spring/optical_flow_saves/2_layer_trials0to32.csv', delimiter=',')
restored_shape_flow_data = loaded_flow_data.reshape((33, 2, 23, 768, 1024)) # edit num_trials (first shape param) to 33 or 34

# Verify the reshaped data
print('Shape of reshaped data:', restored_shape_flow_data.shape)
print(np.all(flow_data == restored_shape_flow_data))

Shape of reshaped data: (33, 2, 23, 768, 1024)
True
